In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/olastor/german-word-frequencies/refs/heads/main/opensubtitles/opensubtitles_cistem_freq.csv"
df = pd.read_csv(url)
# df.to_csv("opensubtitles_cistem_freq.csv", index=False)  # Save it locally if needed
df.sort_values(by="freq", ascending=False, inplace=True)


In [6]:
import sys
import os
# Add the RQ1 directory to the path
sys.path.append(os.path.abspath("../"))
from classification import WordNonwordClassifier

# model_name = "google/gemma-3-12b-it"
# model_name = "google/gemma-3-12b-pt"
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "Tower-Babel/Babel-9B-Chat"
word_nonword_cls = WordNonwordClassifier("English", model_name) # language is not used in the model name, but it is required by the class

Using device: cuda


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [11]:
from tqdm import tqdm
tokenized_words = []
for word in tqdm(df['word']):
    if pd.isna(word):
        tokenized_words.append([])  # or ['<unk>'], or skip, depending on your use case
    else:
        word_str = str(word)
        tokens = word_nonword_cls.tokenizer.tokenize(word_str)
        tokenized_words.append(tokens)
df['word_tokens'] = tokenized_words
df.to_csv("opensubtitles_cistem_freq.csv", index=False)  # Save it locally if needed

100%|██████████| 948638/948638 [00:24<00:00, 38419.19it/s]


In [ ]:
import pandas as pd
df = pd.read_csv("opensubtitles_cistem_freq.csv")

In [15]:
df['word_toens_len'] = df['word_tokens'].apply(lambda x: len(ast.literal_eval(x)))
df['word_toens_len'].value_counts()

word_toens_len
3     242256
4     241229
5     167613
2     132376
6      90097
7      39914
8      16570
9       7024
1       6665
10      2910
11      1128
12       485
13       191
14        89
15        38
16        23
17         7
19         4
20         4
22         3
18         3
0          2
25         2
83         1
28         1
38         1
32         1
21         1
Name: count, dtype: int64

In [35]:
import stanza
nlp = stanza.Pipeline(lang="de", processors="tokenize,pos,lemma", use_gpu=True)  # Initialize Stanza pipeline

2025-06-07 12:06:37 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-06-07 12:06:37 INFO: Downloaded file to /home/hyujang/stanza_resources/resources.json
2025-06-07 12:06:37 WARNING: Language de package default expects mwt, which has been added
2025-06-07 12:06:37 INFO: Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2025-06-07 12:06:37 INFO: Using device: cuda
2025-06-07 12:06:37 INFO: Loading: tokenize
2025-06-07 12:06:37 INFO: Loading: mwt
2025-06-07 12:06:37 INFO: Loading: pos
2025-06-07 12:06:39 INFO: Loading: lemma
2025-06-07 12:06:45 INFO: Done loading processors!


In [36]:
from tqdm import tqdm
df_single_token = df[(df['word_toens_len']==1) & (df['freq'] != 1)].reset_index(drop=True)
stanza_results = []
for word in tqdm(df_single_token['word']):
    try:
        doc = nlp(word)
        stanza_results.append(doc.sentences)
    except Exception as e:
        print(f"Error processing word '{word}': {e}")
        stanza_results.append(None)
df_single_token['stanza'] = stanza_results

In [ ]:
print(df_single_token['stanza'].apply(len).value_counts())
df_single_token['upos_first'] = df_single_token['stanza'].apply(
    lambda sent_list: sent_list[0].words[0].upos if sent_list and sent_list[0].words else None
)
df_single_token = df_single_token[~df_single_token['word'].str.contains(r'[^a-zA-ZäöüÄÖÜß]', na=False)]

stanza
1    6151
Name: count, dtype: int64

In [98]:
df_single_token['upos_first'].value_counts()

upos_first
X        3072
PROPN    1712
NOUN      636
ADJ       414
VERB      130
ADV        64
PUNCT      28
ADP        27
PRON       20
NUM        14
INTJ       14
AUX         9
DET         7
CCONJ       3
SCONJ       1
Name: count, dtype: int64

In [115]:
lang = "German"
model_name = "Tower-Babel/Babel-9B-Chat"
df_single_token[~df_single_token['upos_first'].isin(["PROPN","X"])].reset_index(drop=True).to_csv(f"/home/hyujang/multilingual-inner-lexicon/data/RQ1/WordIdentity/opensubtitles_single_token_list_{model_name.split('/')[-1]}_{lang}.csv", index=False)